In [10]:
import pandas as pd
import numpy as np
np.int=int
np.object=object
np.bool=bool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
import tensorflow as tf
import re

In [14]:
data = pd.read_csv(r"C:\Users\monis\Downloads\Preprocessed Fake Reviews Detection Dataset.csv")

In [15]:
texts = data['text_'].astype(str)
labels = data['label']


In [16]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [17]:
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [42]:
max_sequence_length = 200  #
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [137]:
X_train

array([[   0,    0,    0, ...,   16,   82,  294],
       [   0,    0,    0, ...,    1,  131,  392],
       [   0,    0,    0, ...,  375,    9,  363],
       ...,
       [   0,    0,    0, ..., 4484, 1666,  595],
       [   0,    0,    0, ...,  491,    3,  260],
       [   0,    0,    0, ...,   35,  356,  701]])

In [49]:
encoded_labels[5]

0

In [95]:
class StopOnAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, accuracy_threshold=0.99):
        super(StopOnAccuracy, self).__init__()
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        # Check the training accuracy
        train_accuracy = logs.get('accuracy')
        if train_accuracy and train_accuracy >= self.accuracy_threshold:
            print(f"\nStopping training as training accuracy reached {train_accuracy:.2f}")
            self.model.stop_training = True
stop_on_accuracy = StopOnAccuracy(accuracy_threshold=0.99)

In [85]:
model = Sequential()

model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length))

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [96]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model

# Define the input layer
inputs = Input(shape=(max_sequence_length,))  # Assuming max_sequence_length is defined

# Add the embedding layer
x = Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length)(inputs)

# Add the Conv1D layer
x = Conv1D(filters=128, kernel_size=5, activation='relu')(x)

# Add the MaxPooling1D layer
x = MaxPooling1D(pool_size=2)(x)

# Add the Dense and Dropout layers
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Add the output layer
outputs = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [47]:
model.save("faverev.h5")

In [98]:
# Define the input layer
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
x = Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length)(input_layer)

# Convolutional and pooling layers
x = Conv1D(filters=128, kernel_size=5, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)

# Bidirectional LSTM layer
#x = Bidirectional(LSTM(64))(x)

# Dense and Dropout layers
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
output_layer = Dense(1, activation='sigmoid')(x)

In [76]:
model.save("fakerevs.h5")

In [53]:
# Rebuild the model if needed (or load it)
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_sequence_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [54]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [55]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test),callbacks=[stop_on_accuracy])

Epoch 1/15
1011/1011 [==============================] - 18s 15ms/step - loss: 0.2595 - accuracy: 0.8890 - val_loss: 0.2347 - val_accuracy: 0.9091
Epoch 2/15
1011/1011 [==============================] - 16s 16ms/step - loss: 0.1395 - accuracy: 0.9471 - val_loss: 0.2113 - val_accuracy: 0.9157
Epoch 3/15
1011/1011 [==============================] - 17s 17ms/step - loss: 0.0793 - accuracy: 0.9715 - val_loss: 0.2488 - val_accuracy: 0.9269
Epoch 4/15
1011/1011 [==============================] - 17s 16ms/step - loss: 0.0435 - accuracy: 0.9847 - val_loss: 0.3082 - val_accuracy: 0.9223
Epoch 5/15
1011/1011 [==============================] - 17s 16ms/step - loss: 0.0315 - accuracy: 0.9892 - val_loss: 0.3113 - val_accuracy: 0.9230
Epoch 6/15
1011/1011 [==============================] - 17s 17ms/step - loss: 0.0169 - accuracy: 0.9939 - val_loss: 0.4616 - val_accuracy: 0.9111

Stopping training as training accuracy reached 0.99


In [56]:
model.save("fakerev.h5")

In [57]:
import numpy as np

# Save the model's weights
weights = model.get_weights()

# Convert weights to a binary format
with open('faker.bin', 'wb') as f:
    for w in weights:
        np.save(f, w)


In [58]:
import numpy as np

# Load the weights from the binary file
loaded_weights = []
with open('faker.bin', 'rb') as f:
    for _ in range(len(model.get_weights())):
        loaded_weights.append(np.load(f))

# Set the loaded weights back into the model
model.set_weights(loaded_weights)


In [61]:
print(tf.__version__)

2.6.0


In [23]:
model=load_model('fakerev.h5')

In [5]:
text="well receiv comfort gave gift dialysi patient"

In [11]:
def clean_stopwords(text):
    stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
                    'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before',
                    'being', 'below', 'between', 'both', 'by', 'can', 'd', 'did', 'do',
                    'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from',
                    'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
                    'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
                    'into', 'is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
                    'me', 'more', 'most', 'my', 'myself', 'now', 'o', 'of', 'on', 'once',
                    'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'own', 're',
                    's', 'same', 'she', "shes", 'should', "shouldve", 'so', 'some', 'such',
                    't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
                    'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
                    'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was',
                    'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom',
                    'why', 'will', 'with', 'won', 'y', 'you', "youd", "youll", "youre",
                    "youve", 'your', 'yours', 'yourself', 'yourselves']
    STOPWORDS = set(stopwordlist)
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def clean_punctuations(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def clean_repeating_characters(text):
    return re.sub(r'(.)\1+', r'\1', text)

def clean_URLs(text):
    return re.sub(r"((www\.[^\s]+)|(http\S+))", "", text)

def clean_numeric(text):
    return re.sub('[0-9]+', '', text)

In [18]:
text = text.lower()
text = clean_stopwords(text)
# text = clean_punctuations(text)
text = clean_repeating_characters(text)
text = clean_URLs(text)
text = clean_numeric(text)
sequences = tokenizer.texts_to_sequences([text])

In [19]:
tokenizer = Tokenizer(num_words=10000)  
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

In [20]:
max_sequence_length = 200  #
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [21]:
classify={1:"Computer Genarated",0:"Original Resource"}
text

'wel receiv comfort gave gift dialysi patient'

In [24]:
prediction = model.predict(padded_sequences)
predicted_class = (prediction > 0.76).astype(int)
print(f"Predicted class: {classify[predicted_class[0][0]]}")
prediction

2/2 [==============================] - 3s 33ms/step
Predicted class: Original Resource


array([[0.7348745 ],
       [0.74785995],
       [0.751805  ],
       [0.76074433],
       [0.75550544],
       [0.74785995],
       [0.7579335 ],
       [0.74785995],
       [0.74892724],
       [0.7611296 ],
       [0.76074433],
       [0.7579335 ],
       [0.73555   ],
       [0.75698996],
       [0.7603529 ],
       [0.73555   ],
       [0.75550544],
       [0.76001155],
       [0.76074433],
       [0.7177834 ],
       [0.76193124],
       [0.7611296 ],
       [0.74785995],
       [0.76074433],
       [0.7177834 ],
       [0.74892724],
       [0.7603529 ],
       [0.76001155],
       [0.76074433],
       [0.7563053 ],
       [0.74892724],
       [0.76193124],
       [0.751805  ],
       [0.74208796],
       [0.7372752 ],
       [0.74892724],
       [0.76074433],
       [0.76477313],
       [0.76193124],
       [0.76001155],
       [0.74892724],
       [0.74785995],
       [0.74360526],
       [0.76001155]], dtype=float32)